## קוד מבוא

In [58]:
import os
import pathlib
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely import wkt
from matplotlib import pyplot as plt 

import folium
import fiona
from mpl_toolkits.axes_grid1 import make_axes_locatable
from shapely.geometry import Point


In [59]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

## פונקציות

In [60]:
def double_taz_num(df):
    dup_taz_num=df.groupby(['Taz_num']).size().reset_index(name='count').query('count>1').Taz_num.to_list()
    return df.loc[df['Taz_num'].isin(dup_taz_num)]

In [61]:
def make_point(df):
    df_point=df.copy()
    df_point['centroid'] = df_point.representative_point()
    df_point=df_point.set_geometry('centroid')
    df_point=df_point.drop(columns=['geometry'],axis=1)
    return df_point

In [62]:
def up_load_gdb(path,layer_name):
    path='{}'.format(path)
    layer_list=fiona.listlayers(path)
    gpd_layer=gpd.read_file(path, layer=layer_list.index(layer_name))
    return gpd_layer

In [63]:
def up_load_shp(path):
    path='{}'.format(path)
    gpd_layer=gpd.read_file(path)
    return gpd_layer

In [64]:
def unique_id_in_one_taz(df_to_geoode,unique_field,gpd_for_geocode,taz):
    
    code_to_find=list(df_to_geoode[unique_field].unique())

    gpd_for_geocode_to_sum_by_taz=gpd_for_geocode[[unique_field,'geometry']].loc[gpd_for_geocode[unique_field].isin(code_to_find)]

    gpd_for_geocode_to_sum_by_taz=gpd.sjoin(taz[['Taz_num','geometry']],gpd_for_geocode_to_sum_by_taz)

    gpd_for_geocode_in_one_taz=list(gpd_for_geocode_to_sum_by_taz.groupby(unique_field)[['Taz_num']].nunique().query('Taz_num==1').reset_index()[unique_field])

    return gpd_for_geocode_in_one_taz

In [65]:
def up_load_df(folder_path,file_name):
    
    path_df=r'{}\{}.xlsx'.format(folder_path,file_name)
    df=pd.read_excel(path_df)
    df=df.dropna(how='all')

    return df

In [66]:
def find_files_with_pattern(folder_path, pattern):
    """
    Find files in a directory that match a certain pattern.
    
    Args:
    - directory (str): The directory path.
    - pattern (str): The pattern to search for in file names.
    
    Returns:
    - List of file paths matching the pattern.
    """
    files = []
    for root, _, filenames in os.walk(folder_path):
        for filename in filenames:
            if pattern in filename:
                files.append(os.path.join(root, filename))
    return files

## code

### עיבוד חומר מקדים

In [67]:
#העלת משתנים להרצת הקוד
df_inputs_outputs = pd.read_excel('inputs_outputs.xlsx')

software_data_folder_location=df_inputs_outputs['location'][0]

forecast_version_folder_location=df_inputs_outputs['location'][1]


In [68]:
v_date='230720'

In [69]:
path=r'{}\background_files'.format(software_data_folder_location)

In [70]:
v_date='230720'

In [72]:
taz_2020_catalog=up_load_df(path,'forecast_2020_{}_age_dis_type'.format(v_date))

matching_files=find_files_with_pattern(r'{}'.format(forecast_version_folder_location), '2020_jtmt_forcast_full')
matching_files[0]

file = pathlib.Path(matching_files[0])
file_name = file.name
folder = str(file.parent)
file_name[:-5]

taz_2020_aprt=up_load_df(forecast_version_folder_location, file_name[:-5])

,Taz_num,Agg_taz_nu,Taz_name,Muni_Heb,jeru_metro,zonetype,in_jerusal,main_secto,aprt_20,student_dorms,student_yeshiva_with_add_from_old,hh,pop_without_dorms_yeshiva,pop,hh_size,pop_emp_employed,commuting,student_toddlers,student_gov,cbs_muni_student_left_by_pre_of_demand_left,cbs_muni_student_left_by_pre_of_demand,student_chardi_not_gov,student_arab_not_gov,student,uni_students,student_yeshiva,yeshiva_dorms_pop_15,yeshiva_dorms_pop_20,yeshiva_dorms_pop_25,yeshiva_dorms_pop_sum,add_from_old_student_yeshiva,add_from_kollim_demand,emp_from_uni_student,emp_from_Yeshiva_student,emp_from_student,emp_Education,okev,emp_okev,emp_not_okev,total_emp,agri,Indus,Com_hotel,Business,Public
0,6006,60,נוה יעקב - מרכז מסחרי,ירושלים,1,Jerusalem,yes,U_Orthodox,809.000000,0,356,820.000000,4753.000000,4764.000000,5.875155,1058.512500,0.05,320.100000,714,0.000000,0.000000,0,0,1354.200000,0.00,356,8,3,0,11,0,0.000000,0.000,7,197.117904,204.117904,1,267.779000,0.000000,471.896904,0.000000,0.000000,174.056350,53.555800,40.166850
1,6007,60,נוה יעקב תחתון,ירושלים,1,Jerusalem,yes,U_Orthodox,846.000000,0,96,846.000000,4025.000000,4025.000000,4.757683,925.069600,0.05,242.400000,275,0.000000,0.000000,0,0,759.800000,0.00,96,0,0,0,0,0,211.550000,0.000,3,110.596798,113.596798,1,280.026000,0.000000,393.622798,0.000000,0.000000,168.015600,56.005200,56.005200
2,5807,58,רמות - בריכת מים,ירושלים,1,Jerusalem,yes,U_Orthodox,823.000000,0,665,1116.000000,5708.000000,6001.000000,6.935601,1368.941600,0.05,233.700000,4119,0.000000,0.000000,0,0,4586.400000,675.00,665,195,89,9,293,0,0.000000,101.250,22,667.598253,790.848253,1,272.413000,0.000000,1063.261253,0.000000,0.000000,136.206500,81.723900,54.482600
3,5806,58,רמות פולין,ירושלים,1,Jerusalem,yes,U_Orthodox,912.000000,0,3949,1719.000000,6006.000000,6813.000000,6.585526,1401.426900,0.05,261.000000,2947,0.000000,0.000000,0,0,3469.000000,0.00,3949,539,244,24,807,0,0.000000,0.000,106,504.949054,610.949054,0,0.000000,0.000000,610.949054,0.000000,0.000000,0.000000,0.000000,0.000000
4,5802,58,רמות - הדף היומי,ירושלים,1,Jerusalem,yes,U_Orthodox,828.000000,0,0,828.000000,3781.000000,3781.000000,4.566425,900.480100,0.05,203.700000,221,0.000000,0.000000,0,0,628.400000,0.00,0,0,0,0,0,0,311.600000,0.000,0,91.470160,91.470160,1,274.068000,0.000000,365.538160,0.000000,0.000000,137.034000,82.220400,54.813600
5,6401,64,מחנה פליטים שועפאת - מזרח,ירושלים,1,Jerusalem,yes,arabs_behined_seperation_wall,5273.775484,0,0,5273.775484,29005.765163,29005.765163,5.500000,5787.944434,0.05,1233.455004,187,0.000000,3507.390349,0,442,6603.300358,0.00,0,0,0,0,0,0,0.000000,0.000,0,961.179091,961.179091,1,2536.686008,0.000000,3497.865099,0.000000,0.000000,1522.011605,507.337202,507.337202
6,5703,57,רמות - גן טכנולוגיה,ירושלים,1,Jerusalem,yes,U_Orthodox,731.000000,0,13,731.000000,2629.000000,2629.000000,3.596443,662.645800,0.05,131.100000,1098,0.000000,0.000000,0,0,1360.200000,0.00,13,0,0,0,0,0,203.400000,0.000,1,197.991266,198.991266,1,241.961000,0.000000,440.952266,0.000000,0.000000,157.274650,48.392200,36.294150
7,5706,57,רמות - גן העץ,ירושלים,1,Jerusalem,yes,U_Orthodox,723.000000,0,95,723.000000,3761.000000,3761.000000,5.201936,925.059900,0.05,193.800000,391,0.000000,0.000000,0,0,778.600000,0.00,95,0,0,0,0,0,219.500000,0.000,2,113.333333,115.333333,0,0.000000,0.000000,115.333333,0.000000,0.000000,0.000000,0.000000,0.000000
8,5105,51,סנהדריה המורחבת,ירושלים,1,Jerusalem,yes,U_Orthodox,857.000000,0,1002,1306.000000,4356.000000,4805.000000,5.082847,932.121500,0.05,174.300000,5006,0.000000,0.000000,0,0,5354.600000,0.00,1002,320,118,11,449,0,0.000000,0.000,40,779.417758,819.417758,0,0.000000,0.000000,819.417758,0.000000,0.000000,0.000000,0.000000,0.000000
9,5210,52,קרית צאנז,ירושלים,1,Jerusalem,yes,U_Orthodox,876.000000,0,1328,1251.000000,4029.600000,4404.600000,4.600000,931.338290,0.05,164.828864,1792,0.000000,0.000000,0,0,2121.657729,875.00,1328,276,90,9,375,0,0.000000,131.250,46,308.829364,486.079364,0,0.000000,0.000000,486.079364,0.000000,0.000000,0.00000

In [ ]:
save_file=r'W:\Projects\הסעת המונים\01_שלב ה\קבצי עבודה\תחזיות_דמוגרפיות\תחזיות_2050\jtmt\jew_kibolet\output'

kibolt_jew_2050=up_load_df(save_file,'{}_kibolt_jew_till_2050'.format(v_date)).set_index('Taz_num')

In [105]:
age_catalog=up_load_df(r'W:\Projects\הסעת המונים\01_שלב ה\קבצי עבודה\תחזיות_דמוגרפיות\תחזיות_2050\התפלגות גילים\backround_files','age_des_types')

taz_with_aprt_diversity=up_load_shp(r'W:\Data\Forecast\creat_age_distribution\v_1\backround_files\shp\cost_diversity_for_age_distribution.shp').set_index('Taz_num')

taz_2020_catalog=taz_2020_catalog.merge(taz_2020_aprt[['Taz_num','zonetype','aprt_20','pop_without_dorms_yeshiva','Taz_name','Muni_Heb']],on='Taz_num',how='left').set_index('Taz_num')

taz_2020_catalog['taz_with_aprt_diversity']=taz_with_aprt_diversity['div']

taz_2020_catalog['taz_expensive']=taz_with_aprt_diversity['cost']

col=['classification_name','age_cycle']

taz_2020_catalog=taz_2020_catalog.reset_index().merge(age_catalog[col],on='classification_name',how='left').set_index('Taz_num')

taz_2020_catalog=taz_2020_catalog.join(kibolt_jew_2050).query('sector_for_age != "Arab"')

taz_2020_catalog=taz_2020_catalog.fillna(0)

## חישוב 

In [106]:
singal_year=list(range(2025,2051,5))

In [107]:
years=['add_aprt_2020_2025',
 'add_aprt_2025_2030',
 'add_aprt_2030_2035',
 'add_aprt_2035_2040',
 'add_aprt_2040_2045',
 'add_aprt_2045_2050']

y_before=['2020','2020_2025',
 '2025_2030',
 '2030_2035',
 '2035_2040',
 '2040_2045']

only_years=['2020_2025',
 '2025_2030',
 '2030_2035',
 '2035_2040',
 '2040_2045',
 '2045_2050']

In [108]:
taz_2020_catalog['age_cycle_time']=0
taz_2020_catalog['current_aprt']=taz_2020_catalog['aprt_20']
years_to_change_cycle_stage=20
age_cycle_limit=4
taz_2020_catalog['age_cycle_2020']=taz_2020_catalog['age_cycle']
taz_2020_catalog['changed_this_cycle']=0

In [109]:
for y, y1 in zip(years, only_years):
    
        
    
    #### אזורים ללא יח"ד קיימות בשנת הבסיס מתאכלסים בהתפלגות גילים הראשוני

    taz_with_aprt_diversity_condition = taz_2020_catalog['taz_with_aprt_diversity']==1
    taz_expensive_condition = taz_2020_catalog['taz_expensive']==1
    

    taz_2020_catalog.loc[
        (taz_2020_catalog['current_aprt']==0)&
        (taz_2020_catalog['{}'.format(y)]>0)&
        (taz_2020_catalog['changed_this_cycle']==0)&
        taz_with_aprt_diversity_condition,['age_cycle','age_cycle_time','explanation_{}'.format(y1),'changed_this_cycle']
    ]=[3,0,' | new diverse neighborhood | ',1]

    
    taz_2020_catalog.loc[
        (taz_2020_catalog['current_aprt']==0)&
        (taz_2020_catalog['{}'.format(y)]>0)&
        (taz_2020_catalog['changed_this_cycle']==0)&
        taz_expensive_condition&
        ~taz_with_aprt_diversity_condition,['age_cycle','age_cycle_time','explanation_{}'.format(y1),'changed_this_cycle']
    ]=[2,0,' | new non-diverse expensive neighborhood  | ',1]
    
    taz_2020_catalog.loc[
        (taz_2020_catalog['current_aprt']==0)&
        (taz_2020_catalog['{}'.format(y)]>0)&
        (taz_2020_catalog['changed_this_cycle']==0)&
        ~taz_expensive_condition&
        ~taz_with_aprt_diversity_condition,['age_cycle','age_cycle_time','explanation_{}'.format(y1),'changed_this_cycle']
    ]=[1,0,' | new non-diverse neighborhood  | ',1]

    #### אזורים עם יח"ד קיימות בשנת הבסיס מתאכלסים בהתפלגות גילים כפונצקיה של יחס קיבולת למצב הקיים

    taz_2020_catalog.loc[
        (taz_2020_catalog['current_aprt']>0),'precent_add_to_current'
    ]=taz_2020_catalog['{}'.format(y)]/taz_2020_catalog['current_aprt']


    
    
    
    #### יחס נמוך - הקיבולת בטלה ביחס למצב הקיים
        #### יחס גדול - הקיבולת מבטלת את המצב הקיים

    kibolet_condition = taz_2020_catalog['precent_add_to_current'] > 0.5
    yush_condition =  taz_2020_catalog['zonetype']=='Judea and Samaria'

    taz_2020_catalog.loc[
        kibolet_condition &
        ~yush_condition &
        (taz_2020_catalog['changed_this_cycle']==0)&
        taz_with_aprt_diversity_condition,['age_cycle','age_cycle_time','explanation_{}'.format(y1),'changed_this_cycle']
    ]=[3,0,'| renew diverse neighborhood |',1]
    
    taz_2020_catalog.loc[
        kibolet_condition &
        taz_expensive_condition &
        (taz_2020_catalog['changed_this_cycle']==0)&
        ~taz_with_aprt_diversity_condition,['age_cycle','age_cycle_time','explanation_{}'.format(y1),'changed_this_cycle']
    ]=[2,0,'| a lot of new aprts in non-diverse expensive neighborhood|',1]

    taz_2020_catalog.loc[
        kibolet_condition &
        (taz_2020_catalog['changed_this_cycle']==0)&
        ~taz_expensive_condition &
        ~taz_with_aprt_diversity_condition,['age_cycle','age_cycle_time','explanation_{}'.format(y1),'changed_this_cycle']
    ]=[1,0,'| a lot of new aprts in non-diverse neighborhood|',1]

    #### יחס בינוני

    ##### ביוש אין התחדשות עירונית לכן אם יחס גבוהה יחסית זה כמו בנייה חדשה עם אכלוס של סוג התפלגות גילים הראשוני

    
    kibolet_condition =  taz_2020_catalog['precent_add_to_current']>= 0.20

    taz_2020_catalog.loc[
        (yush_condition)&
        (taz_2020_catalog['changed_this_cycle']==0)&
        (kibolet_condition) , ['age_cycle','age_cycle_time','explanation_{}'.format(y1),'changed_this_cycle']
    ] = [1, 0,'| a lot of new aprts in non-diverse neighborhood in yush |',1]


    ##### יוש אין התחדשות עירונית לכן אם יחס לא גבוהה יחסית זה מונע התבגרות של המצב הקיים
    

    ##### אם לא ביוש אז יש אפשרות התחדשות עירונית לכן המקום נהיה מגוון

    
    kibolet_condition =  (taz_2020_catalog['precent_add_to_current']>= 0.25) & (taz_2020_catalog['precent_add_to_current'] <= 0.5)
    

    taz_2020_catalog.loc[
        ~(yush_condition)&
        (kibolet_condition)&
        (taz_with_aprt_diversity_condition) , ['age_cycle','explanation_{}'.format(y1)]
    ] =  [3,'| a lot of new aprts in diverse neighborhood|']
    
    
    
    #השעון התקדם כבר ולא צריך לשנות כלום

    #### קוד לתהליך התבגרות התפלגות הגילים כפונקציה של סוג מגזר

    

    age_cycle_condition = taz_2020_catalog['age_cycle']<age_cycle_limit

    age_cycle_time_condition=(taz_2020_catalog['age_cycle_time']%years_to_change_cycle_stage==0)&(taz_2020_catalog['age_cycle_time']>0)

    sector_condition=taz_2020_catalog['sector_for_age']=='hitnachlut' 
    
    taz_2020_catalog.loc[
        (age_cycle_time_condition) &
        (sector_condition) &
        (taz_2020_catalog['changed_this_cycle']==0)&
        (age_cycle_condition),['explanation_{}'.format(y1),'changed_this_cycle']
    ]=['| natural aging |',1]
    
    taz_2020_catalog.loc[
        (age_cycle_time_condition) &
        (sector_condition) &
        (age_cycle_condition),'age_cycle'
    ]=taz_2020_catalog['age_cycle']+1
    
    
    taz_2020_catalog.loc[
        (age_cycle_time_condition) &
        (sector_condition) &
        (age_cycle_condition),'age_cycle_time'
    ]=taz_2020_catalog['age_cycle_time']+5

    sector_condition=taz_2020_catalog['sector_for_age']=='Jewish'

    age_cycle_condition = taz_2020_catalog['age_cycle']==1


    taz_2020_catalog.loc[
        (age_cycle_time_condition) &
        (taz_2020_catalog['changed_this_cycle']==0)&
        (sector_condition) &
        (age_cycle_condition),['age_cycle','explanation_{}'.format(y1),'changed_this_cycle']
    ]=[2 ,'| natural aging |',1]
    
    
    taz_2020_catalog.loc[
        (age_cycle_time_condition) & 
        (sector_condition) &
        (age_cycle_condition),'age_cycle_time'
    ]=taz_2020_catalog['age_cycle_time']+5


    sector_condition=taz_2020_catalog['sector_for_age']=='Jewish'

    age_cycle_condition = taz_2020_catalog['age_cycle']==2

    

    taz_2020_catalog.loc[
        (age_cycle_time_condition) &
        (age_cycle_condition) &
        (sector_condition) &
        (taz_2020_catalog['changed_this_cycle']==0) &
        (taz_with_aprt_diversity_condition),['age_cycle','explanation_{}'.format(y1),'changed_this_cycle']
    ]=[3,'| natural aging to a diverse neighborhood |',1]
    
    taz_2020_catalog.loc[
        (age_cycle_time_condition) &
        (age_cycle_condition) &
        (sector_condition) &
        (taz_with_aprt_diversity_condition),'age_cycle_time'
    ]=taz_2020_catalog['age_cycle_time']+5

    sector_condition=taz_2020_catalog['sector_for_age']=='Jewish'

    age_cycle_condition = taz_2020_catalog['age_cycle']==4

    

    taz_2020_catalog.loc[
        (age_cycle_time_condition) &
        (age_cycle_condition) &
        (sector_condition) &
        (taz_2020_catalog['changed_this_cycle']==0) &
        (taz_with_aprt_diversity_condition),['age_cycle','explanation_{}'.format(y1),'changed_this_cycle']
    ]=[3,'| old to a diverse neighborhood |',1]
    
    taz_2020_catalog.loc[
        (age_cycle_time_condition) &
        (age_cycle_condition) &
        (sector_condition) &
        (taz_with_aprt_diversity_condition),'age_cycle_time'
    ]=taz_2020_catalog['age_cycle_time']+5
   
    
    sector_condition=taz_2020_catalog['sector_for_age']=='U_Orthodox'

    age_cycle_condition = taz_2020_catalog['age_cycle']==1


    taz_2020_catalog.loc[
        (age_cycle_time_condition) &
        (sector_condition) &
        (taz_2020_catalog['changed_this_cycle']==0) &
        (age_cycle_condition),['age_cycle','explanation_{}'.format(y1),'changed_this_cycle']
    ]=[2 ,'| natural aging |',1]
    
    
    taz_2020_catalog.loc[
        (age_cycle_time_condition) &
        (sector_condition) &
        (age_cycle_condition),'age_cycle_time'
    ]=taz_2020_catalog['age_cycle_time']+5

    age_cycle_condition = taz_2020_catalog['age_cycle']==2

    taz_2020_catalog.loc[
        (age_cycle_time_condition) &
        (taz_2020_catalog['changed_this_cycle']==0) &
        (age_cycle_condition) &
        (sector_condition) &
        (taz_with_aprt_diversity_condition),['age_cycle','explanation_{}'.format(y1),'changed_this_cycle']
    ]=[3,'| natural aging to a diverse neighborhood |',1]
    
    
    taz_2020_catalog.loc[
        (age_cycle_time_condition) &
        (age_cycle_condition) &
        (sector_condition) &
        (taz_with_aprt_diversity_condition),'age_cycle_time'
    ]=taz_2020_catalog['age_cycle_time']+5
    
    
    taz_2020_catalog.loc[
        (age_cycle_time_condition) &
         (taz_2020_catalog['changed_this_cycle']==0) &
        (age_cycle_condition) &
        (sector_condition) &
        ~(taz_with_aprt_diversity_condition),['age_cycle','explanation_{}'.format(y1),'changed_this_cycle']
    ]=[4,'| natural aging |',1]
    
    age_cycle_condition = taz_2020_catalog['age_cycle']==4
    
    taz_2020_catalog.loc[
        (age_cycle_time_condition) &
        (taz_2020_catalog['changed_this_cycle']==0) &
        (age_cycle_condition) &
        (sector_condition) &
        (taz_with_aprt_diversity_condition),['age_cycle','explanation_{}'.format(y1),'changed_this_cycle']
    ]=[3,'| old to a diverse neighborhood |',1]
    
    
    taz_2020_catalog.loc[
        (age_cycle_time_condition) &
        (age_cycle_condition) &
        (sector_condition) &
        (taz_with_aprt_diversity_condition),'age_cycle_time'
    ]=taz_2020_catalog['age_cycle_time']+5
    
    
   

    taz_2020_catalog.loc[
        (age_cycle_time_condition) &
        (age_cycle_condition) &
        (sector_condition) &
        ~(taz_with_aprt_diversity_condition),'age_cycle_time'
    ]=taz_2020_catalog['age_cycle_time']+5

    taz_2020_catalog.loc[(taz_2020_catalog['changed_this_cycle']==0),'age_cycle_time'
    ]=taz_2020_catalog['age_cycle_time']+5
    
    taz_2020_catalog['changed_this_cycle']=0

    
    taz_2020_catalog['current_aprt']=taz_2020_catalog['current_aprt']+taz_2020_catalog['{}'.format(y)]
    taz_2020_catalog['current_aprt_{}'.format(y1)]= taz_2020_catalog['current_aprt']
    taz_2020_catalog['age_cycle_{}'.format(y1)]= taz_2020_catalog['age_cycle']
    taz_2020_catalog['age_cycle_time_{}'.format(y1)]= taz_2020_catalog['age_cycle_time']
    taz_2020_catalog['precent_add_to_current_{}'.format(y1)]= taz_2020_catalog['precent_add_to_current']

In [110]:
file_date=pd.Timestamp.today().strftime('%y%m%d')

## ייצוא

In [111]:
taz_2020_catalog['hh_size_2020']=taz_2020_catalog['pop_without_dorms_yeshiva']/taz_2020_catalog['aprt_20']

In [112]:
for y1,sy,y_b, in zip(only_years, singal_year,y_before):
    taz_2020_catalog.loc[(taz_2020_catalog['age_cycle_{}'.format(y1)]==taz_2020_catalog['age_cycle_{}'.format(y_b)])&(taz_2020_catalog['age_cycle_{}'.format(y1)]==3),'hh_size_{}'.format(y1)]=taz_2020_catalog['hh_size_{}'.format(y_b)]
    taz_2020_catalog.loc[(taz_2020_catalog['age_cycle_{}'.format(y1)]==taz_2020_catalog['age_cycle_{}'.format(y_b)])&(taz_2020_catalog['age_cycle_{}'.format(y1)]==3),'hh_size_{}'.format(y1)]=taz_2020_catalog['hh_size_{}'.format(y1)]+0.15
    

In [113]:
col_age=[ 'pop_0',
 'pop_10',
 'pop_15',
 'pop_20',
 'pop_25',
 'pop_30',
 'pop_35',
 'pop_40',
 'pop_45',
 'pop_5',
 'pop_50',
 'pop_55',
 'pop_60',
 'pop_65',
 'pop_70',
 'pop_75up']

In [114]:
for y1,sy,y_b, in zip(only_years, singal_year,y_before):
    
    col_taz=['Taz_num','current_aprt_{}'.format(y1),'explanation_{}'.format(y1),'age_cycle_{}'.format(y1),'hh_size_{}'.format(y1),'sector_for_age']
    
    left_col=['sector_for_age','age_cycle_{}'.format(y1)]
    right_col=['sector','age_cycle']

    df=taz_2020_catalog.reset_index()[col_taz].merge(age_catalog,left_on=left_col,right_on=right_col,how='left').set_index('Taz_num')
    

    df['pop']=df['current_aprt_{}'.format(y1)]*df['hh_size_{}'.format(y1)]
    df.loc[df['hh_size_{}'.format(y1)].isna(),'pop']=df['current_aprt_{}'.format(y1)]*df['hh_size']
    
        
    df['pop_{}'.format(sy)]=df['pop']
    
    df['aprt_{}'.format(sy)]=df['current_aprt_{}'.format(y1)]
    
    df[col_age]=df[col_age].multiply(df['pop'], axis="index")    
    
    file_save=r'W:\Projects\הסעת המונים\01_שלב ה\קבצי עבודה\תחזיות_דמוגרפיות\תחזיות_2050\jtmt\jew_pop'
    save_excel_path=r'{}\{}_forecast_pop_jewish_{}_jtmt.xlsx'.format(file_save,file_date,y1)
    
    df.to_excel(save_excel_path,index=True)
    
    taz_2020_catalog=taz_2020_catalog.join(df[['pop_{}'.format(sy),'aprt_{}'.format(sy)]])
    
 

In [115]:
taz_2020_catalog['add_aprt_till_2050']=taz_2020_catalog[['add_aprt_2020_2025',
 'add_aprt_2025_2030',
 'add_aprt_2030_2035',
 'add_aprt_2035_2040',
 'add_aprt_2040_2045',
 'add_aprt_2045_2050']].sum(axis=1)

In [116]:
col_order=['Taz_name','Muni_Heb','zonetype',
 'taz_with_aprt_diversity',
 'taz_expensive',
 'sector_for_age',
 'explanation_2020_2025',
 'explanation_2025_2030',
 'explanation_2030_2035',
 'explanation_2035_2040',
 'explanation_2040_2045',
 'explanation_2045_2050',
 'aprt_20','aprt_2025','aprt_2030','aprt_2035','aprt_2040','aprt_2045'
,'aprt_2050', 'add_aprt_2020_2025',
 'add_aprt_2025_2030',
 'add_aprt_2030_2035',
 'add_aprt_2035_2040',
 'add_aprt_2040_2045',
 'add_aprt_2045_2050',
 'add_aprt_till_2050',
 'pop_without_dorms_yeshiva',
 'pop_2025',
 'pop_2030',
 'pop_2035',
 'pop_2040',
 'pop_2045',
 'pop_2050',
 'classification_name',
 'age_cycle_2020',
 'age_cycle_2020_2025',
 'age_cycle_2025_2030',
 'age_cycle_2030_2035',
 'age_cycle_2035_2040',
 'age_cycle_2040_2045',
 'age_cycle_2045_2050',
 'age_cycle_time_2020_2025',
 'age_cycle_time_2025_2030',
 'age_cycle_time_2030_2035',
 'age_cycle_time_2035_2040',
 'age_cycle_time_2040_2045',
 'age_cycle_time_2045_2050',
 'precent_add_to_current_2020_2025',
 'precent_add_to_current_2025_2030',
 'precent_add_to_current_2030_2035',
 'precent_add_to_current_2035_2040',
 'precent_add_to_current_2040_2045',
 'precent_add_to_current_2045_2050']

In [117]:
file_save=r'W:\Projects\הסעת המונים\01_שלב ה\קבצי עבודה\תחזיות_דמוגרפיות\תחזיות_2050\jtmt\jew_pop'
save_excel_path=r'{}\{}_forecast_pop_jewish_till_2050_jtmt.xlsx'.format(file_save,file_date)

In [118]:
taz_2020_catalog[col_order].to_excel(save_excel_path)

In [119]:
taz_2020_catalog_pivot=taz_2020_catalog.pivot_table(index='Muni_Heb',aggfunc=sum)

In [120]:
taz_2020_catalog_pivot['add_pop_till_2050']=taz_2020_catalog_pivot['pop_2050']-taz_2020_catalog_pivot['pop_without_dorms_yeshiva']

In [121]:
taz_2020_catalog_pivot['precent_growth_till_2050']=round(taz_2020_catalog_pivot['pop_2050']/taz_2020_catalog_pivot['pop_without_dorms_yeshiva'],2)

In [122]:
col_order=['pop_without_dorms_yeshiva',
 'pop_2025',
 'pop_2030',
 'pop_2035',
 'pop_2040',
 'pop_2045',
 'pop_2050','add_pop_till_2050','precent_growth_till_2050','aprt_20','aprt_2025','aprt_2030','aprt_2035','aprt_2040','aprt_2045'
,'aprt_2050',
 'add_aprt_2020_2025',
 'add_aprt_2025_2030',
 'add_aprt_2030_2035',
 'add_aprt_2035_2040',
 'add_aprt_2040_2045',
 'add_aprt_2045_2050',
 'add_aprt_till_2050' ]

In [123]:
save_excel_path=r'{}\{}_pivot_by_muni_forecast_pop_jewish_till_2050_jtmt.xlsx'.format(file_save,file_date)

In [124]:
taz_2020_catalog_pivot[col_order].query('index!="0"').to_excel(save_excel_path)